In [3]:
import numpy as np
import os.path as osp
import matplotlib.pylab as plt
import sys
sys.path.insert(0, "/home/davidtseng/")

# from latent_3d_points.src.point_net_ae import PointNetAutoEncoder
# from latent_3d_points.src.autoencoder import Configuration as Conf
# from latent_3d_points.src.neural_net import MODEL_SAVER_ID

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.general_utils import plot_3d_point_cloud
# from latent_3d_points.src.tf_utils import reset_tf_graph

# from latent_3d_points.src.vanilla_gan import Vanilla_GAN
# from latent_3d_points.src.w_gan_gp import W_GAN_GP
# from latent_3d_points.src.generators_discriminators import latent_code_discriminator_two_layers,\
# latent_code_generator_two_layers

In [4]:
all_pc_data = load_all_point_clouds_under_folder("/home/davidtseng/latent_3d_points/data/shape_net_core_uniform_samples_2048", n_threads=2, file_ending='.ply', verbose=True)

/home/davidtseng/latent_3d_points/src/in_out.py:133: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


57449 pclouds were loaded. They belong in 57 shape-classes.


In [7]:
# x = all_pc_data.next_batch(1)


In [14]:
L = all_pc_data.next_batch(21)
L= L[0]
i = 0
plot_3d_point_cloud(L[i][:, 0], L[i][:, 1], L[i][:, 2], in_u_sphere=True);
i = 20
plot_3d_point_cloud(L[i][:, 0], L[i][:, 1], L[i][:, 2], in_u_sphere=True);

AttributeError: 'module' object has no attribute 'to_rgba'